In [1]:
# coding:u8
import json
import numpy as np
import pandas as pd
from tqdm import tqdm as tqdm
import gc
from gensim.models import Word2Vec
from sklearn.model_selection import KFold

import pickle

In [232]:
ner_result = pd.read_pickle('result/ner_bert_result.pkl')
ner_result['text'] = ner_result['text'].apply(lambda x:''.join(x[1:-1]))
ner_result['pos'] = ner_result['pos'].apply(lambda x:[k[0]-1 for k in x])

In [182]:
ner_result.head()

,text,pred,pos
0,张学友一首《当爱变成习惯》很经典,"[张学友, 当爱变成习惯]","[0, 6]"
1,《高庸武侠小说全集》在线阅读,[高庸武侠小说全集],[1]
2,《大话仙侠》三大职业新人必备职业_大话仙,"[大话仙侠, 职业, 新人, 职业]","[1, 8, 10, 14]"
3,《轮回:三界至尊》_黔首的刀著_玄幻,[],[]
4,中国音乐学院 江南 男中音 《隐衷》《黑雾》《 ...,"[中国音乐学院, 江南, 男中音]","[0, 7, 10]"


In [2]:
# 戴安楠ner
ner_result = pd.read_csv('result/el_dev_result.csv',sep='\t')
ner_result['offset'] = ner_result['offset'].apply(lambda x:eval(x.lower()))
ner_result['mention'] = ner_result['mention'].apply(lambda x:eval(x.lower()))
ner_result = ner_result.rename(columns={'mention':'pred','offset':'pos'})

In [50]:
# 原生ner
train_data = []
with open('data/raw_data/train.json', 'r') as f:
    for line in f:
        raw = eval(str(json.loads(line)).lower())
        train_data.append(raw)
originla_train_data = train_data[80000:]

In [3]:
ner_result.head()

,text_id,text,pred,pos
0,1,《忏悔录》 20111016 荒唐出轨,[忏悔录],[1]
1,2,《最终幻想世界》上线steam 11月22日发售,[最终幻想世界],[1]
2,3,《谢文东2》电视剧_全集(1-28集)高清在线观看,"[谢文东2, 电视剧]","[1, 6]"
3,4,进口喜宝奶粉有适合3岁以上的吗,"[进口, 喜宝奶粉, 适合]","[0, 2, 7]"
4,5,“3·11”东日本大地震7周年悼念 仍有约七万人过着...,[东日本大地震],[6]


In [4]:
ner_result.head()

,text_id,text,pred,pos
0,1,《忏悔录》 20111016 荒唐出轨,[忏悔录],[1]
1,2,《最终幻想世界》上线steam 11月22日发售,[最终幻想世界],[1]
2,3,《谢文东2》电视剧_全集(1-28集)高清在线观看,"[谢文东2, 电视剧]","[1, 6]"
3,4,进口喜宝奶粉有适合3岁以上的吗,"[进口, 喜宝奶粉, 适合]","[0, 2, 7]"
4,5,“3·11”东日本大地震7周年悼念 仍有约七万人过着...,[东日本大地震],[6]


In [56]:
ner_result['pos_len'] = ner_result['pos'].apply(lambda x:len(x))
print(sum(ner_result['pos_len']))

27087


In [275]:
ner_result.shape

(10000, 5)

In [6]:
# 包装成和train_data一样的格式
new_train_data = []
for index,row in ner_result.iterrows():
    item = {}
    item['text_id'] = row['text_id']
    item['text'] = row['text']
    item['mention_data'] = []
    for i,p in enumerate(row['pos']):
        item['mention_data'].append({'mention':row['pred'][i], 'offset':str(p)})
    new_train_data.append(item)

In [7]:
new_train_data[0:5]

[{'text_id': 1,
  'text': '《忏悔录》 20111016 荒唐出轨',
  'mention_data': [{'mention': '忏悔录', 'offset': '1'}]},
 {'text_id': 2,
  'text': '《最终幻想世界》上线steam 11月22日发售',
  'mention_data': [{'mention': '最终幻想世界', 'offset': '1'}]},
 {'text_id': 3,
  'text': '《谢文东2》电视剧_全集(1-28集)高清在线观看',
  'mention_data': [{'mention': '谢文东2', 'offset': '1'},
   {'mention': '电视剧', 'offset': '6'}]},
 {'text_id': 4,
  'text': '进口喜宝奶粉有适合3岁以上的吗',
  'mention_data': [{'mention': '进口', 'offset': '0'},
   {'mention': '喜宝奶粉', 'offset': '2'},
   {'mention': '适合', 'offset': '7'}]},
 {'text_id': 5,
  'text': '“3·11”东日本大地震7周年悼念 仍有约七万人过着...',
  'mention_data': [{'mention': '东日本大地震', 'offset': '6'}]}]

In [8]:
# kb['311223']['alias']

In [9]:
train_data = []
train_data_dict = {}
with open('data/raw_data/train.json', 'r') as f:
    for line in f:
        raw = eval(str(json.loads(line)).lower())
        train_data.append(raw)
        train_data_dict[raw['text_id']] = raw
kb = {}
with open('data/raw_data/kb_data', 'r') as f:
    for line in f:
        item = eval(str(json.loads(line)).lower())
        kb[item['subject_id']] = item
# 补充别名实体进入kb
for s in train_data:
    for m in s['mention_data']:
        if m['kb_id'] == 'nil': continue
        if m['mention'] != kb[m['kb_id']]['subject'] and m['mention'] not in kb[m['kb_id']]['alias']:
            #if m['mention']=='网络游戏':
            print(m['mention'],kb[m['kb_id']]['alias'])
            kb[m['kb_id']]['alias'] += [m['mention']]
name_id = {}
for kb_id in kb:
    for item in kb[kb_id]['alias']:
        if item not in name_id:
            name_id[item] = [kb_id]
        else:
            name_id[item].append(kb_id)
    if kb[kb_id]['subject'] not in name_id:
        name_id[kb[kb_id]['subject']] = [kb_id]
    else:
        name_id[kb[kb_id]['subject']].append(kb_id)
for id in name_id:
    name_id[id] = list(set(name_id[id]))

央行 ['中国中央银行', '人民银行', '中国央行', "people's bank of china", "the people's bank of china", '人行', '中国人行']
亿万富翁 ['亿万富豪']
贤秀 ['김현수', 'kim hyeon-soo', '金贤秀']
儿童频道 ['中央少儿频道', '少儿频道', '中央14套', '中央十四套', 'cctv 14', '中央少儿', '央视少儿', '中央儿频道', '电视少儿频道', '少儿电视台']
候红琴 ['hou hongqin', '侯红琴']
唐伯虎点秋香之冲上云霄 ['flirting in the air', '唐伯虎冲上云霄', '唐伯虎上云霄', '冲上云霄唐伯虎']
庆东原•江头即事 ['双调·庆东原·江头即事']
上海 ['上海电视剧频道', '上海电视剧', '上视电视剧', '上视电视剧频道']
月光奏鸣曲 贝多芬 ['月光奏鸣曲']
忘了你忘了我- ['忘了你忘了我']
王杰, ['dave wang', 'wang chieh', '浪子', '孤鹰', '歌坛浪子', '忧郁歌王', '王杰', '港台歌手王杰', '著名歌手王杰']
下载, ['下载']
高新投资集团 ['成都市高新投资集团有限公司']
吓尿 ['吓尿了']
经典名句 ['名句经典']
暴雪 ['暴雪电子娱乐公司', '暴雪公司', '暴雪娱乐', 'blizzard', 'blizzard entertainment']
仙剑 ['天河传', '仙剑奇侠传四', '仙剑奇侠传四']
国家质检总局 ['质检局', '中国国家质量监督检验检疫总局', '中国国家监督检验检疫总局', '国家质监局总局', '中国检疫检验局', '质量检验检疫局', '中国检疫局', '国家检疫检验总局', '国家质量检验监督总局', '国家监督检验检疫总局']
热带风暴 ['热带风暴水上乐园']
农用车 ['auto', 'automobile']
古月哥欠 ['hugo', '胡歌']
守护者的崛起 ['守护者联盟']
票 ['电影票']
李雷和韩 ['李雷和韩梅梅', '李雷和韩梅梅——昨日重现']
李敏德母亲 ['王婉娟', '三姐']
新闻频道 ['cctv 13', '中央13套', '央

圣斗士星矢 圣斗少女翔 ['圣斗少女翔']
英 ['english', '英文']
德 ['德文', 'deutsch']
我不是猪才怪(新) ['我不是猪才怪']
jmedia ['界面']
louis vuitto ['lv', 'louisvuitton', '路易斯威登', '路易·威登', '法国lv', '法国louis vuitton', '路易登威', 'louis', '路易威登lv', 'lv包']
书格 ['红楼梦图咏']
攻略 ['旅游', '游']
死魂曲 新解 ['死魂曲新解']
白鹤洞街 ['白鹤洞街道']
赵丹军 ['主持人']
仙侠情缘 ['剑侠情缘']
大甜甜 ['jing tian', '景甜', '甜甜', '甜儿', '小甜', '公主']
经信委 ['经济信息委委员会']
千禧夜,我们说相声 ['millennium teahouse', '千禧夜', '我们说相声']
党委副书记 ['党委书记', '党工委书记']
古米特·辛格 ['古米特·拉姆·拉希姆·辛格']
判 ['判处']
音乐空间 ['空间音乐']
中国原创音乐基地 ['原创音乐基地']
符文 ['lol文符', '符文lol']
nekopara vol.3 ['nekopara']
谭警官 ['谭乔']
视频 ['尚长荣', '尚叔欣']
心悦会员 ['心悦俱乐部']
穴位 ['墓穴']
vitaly petrov ['维塔利·佩特罗夫']
不眠之夜sleep no more ['不眠之夜']
二中 ['昆山二中', '昆山第二中学', '昆山市二中']
黑暗时代/神剑/亚瑟王神剑 ['excalibur', '黑暗时代', '《黑暗时代》', '黑暗时代excalibur(1981)']
风景园林 ['风景园林专业']
龙之谷2 精灵王座 ['龙之谷2']
1905vip影院 ['1905电影网']
《重装上阵电影》 ['重装上阵']
雷姆.库哈斯 ['rem koolhaas', '雷姆·库哈斯', '库哈斯', 'rem koolhaas', 'remkoolhaas']
大海贼冒险岛魔 ['大海贼冒险岛']
诡计多端2015 ['ร้อยเล่ห์เสน่ห์ร้าย', 'roy rei sa nei rai', '千方百计爱上你', '诡计多端

代表 ['人大代表']
正厅 ['正厅级干部']
客家山歌王 ['客家山歌']
郑州市轨道公司 ['郑州轨道交通有限公司']
和硕简亲王 ['爱新觉罗·费扬武', '芬古', '费扬武']
恶魔总编 ['anna wintour', '安娜·温图尔', 'anna wintour', '安娜温图尔', 'annawintour', '安娜-温图尔', '安娜.温图尔']
吉林省政府 ['吉林政府', '吉林人民政府']
幻影战机 ['法国幻影系列战斗机', 'mirage']
文化 ['开幕']
致敬!国家工程师 ['致敬！国家工程师']
杨钰莹 ['世界以痛吻我']
米谢·罗德里格兹 ['michelle rodriguez', '米歇尔·罗德里格兹', '米歇尔-罗德里格兹', 'michelle rodriguez', '米歇尔罗德里格兹', '米歇尔 罗德里格兹']
音乐 ['charmaine sheh', '佘诗曼', '阿佘', '亚佘', '佘仔', '佘佘', '阿蛇', 'charmaine sheh']
天空体育 ['天空体育台']
伊普斯 ['伊普斯维奇', '伊普斯维奇城足球俱乐部', 'ipswich town football club']
人物像 ['人物']
庭审 ['庭审现场']
邀 ['邀请']
口腔科 ['温州附一', '温一医', '温州一医院', '温州第一附属医院', '温州附属第一医院', '温州附一医', '附一医院', '温一附医院', '温州附一院', '温州医学院附属一医']
国家发改委 ['发改委', '中国国家发展和改革委员会', '中华人民共和国发改委', '国家发展委', '国家改革和发展委员会', '发展委', '发改委员会', '国家发展改革', '国家发展改革委会', '国家发改']
海上钢琴师 ['钢琴家']
古董局中局3·掠宝清单 ['古董局中局3']
《丙申年》生肖猴票 ['丙申年特种邮票', '丙申年猴票']
百鸟 ['song of the phoenix', 'bai niao chao feng', '百鸟朝凤']
总冠军 ['the nba championship', 'nba总冠军']
王黎 ['leon lai ming', '黎明', '金句王', '明福侠', 'leon

大染坊第二部 ['大染坊续', '大染坊2', '大染坊2']
第十三届桃李杯 ['桃李杯', '桃李杯舞蹈比赛', '第十届桃李杯']
云南小龙潭监狱 ['云南省小龙潭监狱']
年轻女性 ['女孩', '女']
性 ['悟性']
河南 ['粉丝', '粉']
中医 ['中医药']
尼娜·哈特利 ['nina hartley', '哈特利']
帮帮我,爱神 ['帮帮我爱神']
副政委 ['政委']
攻略 ['我发明', '我发明爱']
哈农 ['哈农指法', '哈农练指法', '钢琴哈农', '练钢琴指法']
塔季扬娜·塔拉索娃 ['塔拉索娃']
黶庭坚 ['黄庭坚', '黄山谷', '豫章先生', '黄鲁直', '山谷道人', '鲁直', '黄山谷', '山谷']
纳米核心 第三季 ['纳米核心第三季']
泽陂·诗经 ['陈风·泽陂', '泽陂']
世界 ['场景']
零蛋韩剧网 ['韩剧网']
郑医 ['郑州市第五人民医院', '郑州市人民医院']
张国桢教授 ['张国桢']
julio iglesias ['julio lglesias', '胡里奥·伊格莱西亚斯']
bigbang成员 ['최승현', 'choi seung hyun', 't.o.p', '崔胜铉', 't.o.p.', '崔胜贤', '崔胜铉']
终极蜘蛛侠 第一季 ['终极蜘蛛侠第一季']
老袁 ['人称袁项城', '袁宫保', '袁世凯', '袁项城', '洪宪大帝', '袁宫保', '洪宪皇帝', '洪宪帝']
斗罗大陆唐门暗器谱 ['唐门暗器']
寂静 ['silent hill', '沉默之丘', '鬼魅山房', '寂静岭', '鬼魅山房', '沉默之丘', '寂静岭1']
沧州华林集团 ['河北华林酸碱平生物技术有限公司沧州分公司']
沙迪 ['zaid mabrouk', '马布鲁克·扎伊德', '马布鲁克·扎耶德', '马布鲁克·泽耶德']
弗兰茨•约瑟夫一世 ['franz joseph i', '弗兰茨·约瑟夫一世', '弗朗茨·约瑟夫一世', '弗朗茨·约瑟夫一世', 'franz josef i']
圣斗士星矢:斗士之魂 ['圣斗士星矢：斗士之魂']
翻譯 ['翻译', '文言文']
爱的世界 ['ภพรัก', '真爱无界', 'pope ruk']
绯焰地狱 

天津市蓟州区 ['蓟县', '蓟州', '天津市蓟县']
赛微思集团 ['上海赛微思智业投资股份有限公司']
山里寒舍 ['山里寒舍（北京）旅游投资管理有限公司']
科 ['科室']
中国信鸽 ['中信鸽网', '信鸽中国网']
中国环保在线 ['环保在线']
《金牌助理》 ['金牌助理']
兔子帮第二季 ['兔子帮']
落第骑士的英雄谭 ['落第騎士の英雄譚', '落第骑士英雄谭']
天蓝蓝 凤凰传奇 歌曲天蓝蓝mp3 下载 试听 ['天蓝蓝']
我只在乎你 日文珍藏版 ['我只在乎你']
你来自哪颗星国语版 ['넌 어느 별에서 왔니', 'which star are you from', '你来自哪颗星', '公主']
新闻 ['新闻消息']
剧情片电影 ['剧情剧', '剧情', '剧情电影']
高晓松鱼羊野史第6卷 ['鱼羊野史']
秦时丽人 ["the king's woman", '秦时丽人明月心', '丽姬传', '丽姬传', '秦时明月之丽姬传']
戴上手套擦泪:01相遇 ['戴上手套擦泪']
火焰女神 ['仇火女郎']
电 ['电视剧', '电视', '全集', '笑傲江湖', '剧', '电视剧_', '花儿与远方', '电视剧版', '金陵琐事', '电影', '迅雷', '他来了请闭眼', '武间道']
颤抖吧,阿部 ["let's shake it", '颤抖吧', '阿部', 'et', '颤抖吧，阿部', '颤抖吧阿部', '颤抖吧！阿部', '颤抖吧,阿部!']
访 ['采访', '访谈']
宜昌市京剧团 ['湖北省宜昌市京剧团']
帝王史 ['《历代民族和帝王史》']
十大品牌网 ['总裁']
天津妈妈 ['天津妈妈网']
追梦人 ['地址', '价格', '位置地址']
教学指导 ['指导教学']
维秘时尚秀 ['维多利亚的秘密', '维多利亚的秘密秀', '维多利亚的秘密内衣秀', '维密']
果戈里幽灵列车未解之谜 ['幽灵列车']
杭州雷锋夕照 ['雷峰夕照']
党的历史知识简明读本 ['党的基本知识简明读本']
汉马 ['武汉马拉松']
梦 ['做梦']
《屠夫状元》下 ['屠夫状元']
海口秀英区 ['秀英', '英秀区']
驻日美军司令 ['驻日盟军总司令']
《地雷战传奇》续集 ['地雷战传奇之锄奸行动', '新

化学与材料科学学院 ['南京师范大学化学与材料科学学院']
虎与 ['タイガー&ドラゴン', '虎与龙', 'tiger and dragon']
夏波蒂 ['古代中国', '古中国', '中', '联邦论']
杭州不孕不育专科医院 ['杭州不孕不育专科医院有限公司']
土家哭嫁 ['土家族哭嫁']
全国体院功夫全能技大奖赛 ['全国体育院校功夫全能技大奖赛']
中影股份 ['中国电影股份有限公司']
john henry ['john w.henry', '约翰·亨利']
网络isp ['internet service provider', '因特网服务提供商', '因特网服务提供者']
印象 ['湖南省岳阳', '巴陵', '岳阳市']
无弹窗广告 ['无弹窗小说网', '无弹窗']
好好活下去粤语 ['好好活下去']
丈夫 ['老公']
江西宜春 ['月都', '宜春市', '江西省宜春市', 'yichun city', '宜春地区']
建面 ['floor space']
爱玩游戏社区 ['爱玩游戏']
光辉战机 ['光辉战斗机']
经典名句语句 ['语录经典', '经典语句']
妇产科 ['产科']
我曾用心爱过你 ['我曾用心爱着你']
原来 ['shenfen', '楚乔']
林志颖 永不回头 ['永不回头']
百万新娘韩语版 ['파란만장 미스김 10억 만들기', '百万新娘', '金小姐打造10亿韩元计划', '金小姐的10亿韩元计划', '金小姐打造10亿韩元计划']
脚趾头 ['脚趾']
神荼安岩 ['shen tu', '小师叔', '霸道总裁', 'sword boy', '沈图', '神荼']
最坏时候遇见最好的你 ['最坏的时候遇上最好的你']
历史 ['中国历史朝代公元对照简表', '中国朝代', '中国历史朝代表', '中国历代公元对照简表', '朝代特点', '中国朝', '朝代表', '中国历朝']
踢踏舞剧 ['tap dance']
全集 ['a pillow case of mystery', '施公奇案', '施公奇案i', 'a pillow case of mystery', '飞仕影音施公奇案24碟装(vcd)']
美味情缘(国语版) ['a taste of love', '美味情缘']
省管领导干部 ['

In [10]:
len(train_data)

90000

In [11]:
kb_column = []
train_column = []
text_id = []
label_colum = []
num_attrs = []
num_abstract_words = []
num_alias = []
m_id = 0
m_id_list = []
num_candidate = []
equal_subject = []
shuminghao = [] 
entity_common = []
len_mention = []
mention_start = []
is_eng = []
for s in tqdm(new_train_data):
    mention_ner = s['mention_data']
    m_list = []
    for m in mention_ner:
        m_list.append(m['mention'])
    for m in mention_ner:
        if m['mention'] not in name_id:
            continue
        candidate_ids = name_id[m['mention']]
        for m_candidate_id in candidate_ids:
            # 统计配对特征
            candidate_numattrs = 0
            candidate_abstract_numwords = 0
            candidate_detail = kb[m_candidate_id]
            candi_text = ''
            for predicate in candidate_detail['data']:
                candidate_numattrs += 1
                candi_text += predicate['object']
                if predicate['predicate'] == '摘要':
                    candidate_abstract = predicate['object']
                    candidate_abstract_numwords = len(candidate_abstract)
#                 if predicate['predicate'] == '标签':
#                     candidate_label += predicate['object']
            train_column.append(m['mention'])
            kb_column.append(m_candidate_id)
#             if m_candidate_id==m['kb_id']:
#                 label = 1
#             else:
#                 label = 0
#             label_colum.append(label)
            text_id.append(s['text_id'])
            num_attrs.append(candidate_numattrs)
            num_abstract_words.append(candidate_abstract_numwords)
            num_alias.append(len(set(candidate_detail['alias']+[candidate_detail['subject']])))
            m_id_list.append(m_id)
            num_candidate.append(len(candidate_ids))
            equal_subject.append(candidate_detail['subject']==m['mention'])
            shuminghao.append('《'+candidate_detail['subject']+'》' in candidate_abstract)
#             if langid.classify(m['mention'])[0]=='en':
#                 len_mention.append(len(m['mention'].split()))
#                 is_eng.append(1)
#             else:
#                 len_mention.append(len(m['mention']))
#                 is_eng.append(0)
            len_mention.append(len(m['mention']))
            mention_start.append(m['offset'])
            
            i = 0
            for item in m_list:
                if item in candi_text: i+=1
            entity_common.append(i)
        m_id += 1
        
data = pd.DataFrame()
data['text_id'] = text_id
data['kb_id'] = kb_column
data['train_mention'] = train_column
data['num_attrs'] = num_attrs
data['num_abstract_words'] = num_abstract_words
data['num_alias'] = num_alias
data['m_id'] = m_id_list
data['num_candidates'] = num_candidate 
data['mention_equal_subject'] = equal_subject
data['shuminghao'] = shuminghao
data['entity_common']  = entity_common
data['len_mention'] = len_mention
#data['label'] = label_colum
data['mention_start'] = list(map(int,mention_start))

100%|██████████| 10000/10000 [00:01<00:00, 5522.06it/s]


In [12]:
data.shape

(192318, 13)

In [10]:
m_id

27087

In [13]:
# 构建数据配对-只做有label的
kb_column = []
train_column = []
text_id = []
label_colum = []
for s in tqdm(train_data):
    mention_ner = s['mention_data']
    for m in mention_ner:
        if m['mention'] not in name_id:
            continue
        candidate_ids = name_id[m['mention']]
        for m_candidate_id in candidate_ids:
            if m_candidate_id==m['kb_id']:
                label = 1
            else:
                label = 0
            train_column.append(m['mention'])
            kb_column.append(m_candidate_id)
            text_id.append(s['text_id'])
            label_colum.append(label)
        
stat_data = pd.DataFrame()
stat_data['text_id'] = text_id
stat_data['kb_id'] = kb_column
stat_data['train_mention'] = train_column
stat_data['label'] = label_colum


100%|██████████| 90000/90000 [00:01<00:00, 69593.15it/s]


In [14]:
stat_data[stat_data.label==1].shape

(257078, 4)

In [15]:
print(data.shape,stat_data.shape)

(192318, 13) (1834968, 4)


In [16]:
# 流行度特征
# 总流行度
train_group = stat_data.groupby('kb_id',as_index=False)['label'].agg({'label_mean':'mean','label_count':'count'})
data_ctr = data.merge(train_group,on='kb_id',how='left')
print(data_ctr[data_ctr.label_mean.isnull()].shape)

train_group = stat_data.groupby(['kb_id','train_mention'])['label'].agg({'m_label_mean':'mean','m_label_count':'count'})
data_ctr = data_ctr.merge(train_group,on=['kb_id','train_mention'],how='left')

(16164, 15)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  import sys


In [19]:
stat_data.head()

,text_id,kb_id,train_mention,label
0,1,130287,南京南站,0
1,1,311223,南京南站,1
2,1,213561,高铁,0
3,1,341096,高铁,1
4,1,130287,南京南站,0


In [21]:
data_ctr

,text_id,kb_id,train_mention,num_attrs,num_abstract_words,num_alias,m_id,num_candidates,mention_equal_subject,shuminghao,...,rank_num_abstract_words,rank_num_alias,rank_label_mean,rank_label_count,rank_m_label_mean,rank_m_label_count,rank_num_candidates,rank_mention_equal_subject,rank_shuminghao,rank_entity_common
0,1,119304,忏悔录,13,158,1,0,5,True,True,...,4.0,4.5,1.0,3.0,1.0,3.0,3.0,3.0,3.0,3.0
1,1,38456,忏悔录,11,347,4,0,5,True,True,...,2.0,2.0,4.5,3.0,4.5,3.0,3.0,3.0,3.0,3.0
2,1,386314,忏悔录,14,226,2,0,5,True,True,...,3.0,3.0,4.5,3.0,4.5,3.0,3.0,3.0,3.0,3.0
3,1,113234,忏悔录,14,360,8,0,5,True,True,...,1.0,1.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
4,1,372331,忏悔录,5,31,1,0,5,True,True,...,5.0,4.5,2.0,3.0,2.0,3.0,3.0,3.0,3.0,3.0
5,2,303980,最终幻想世界,12,132,1,1,2,True,False,...,1.0,1.5,1.0,1.5,1.0,1.5,1.5,1.5,2.0,2.0
6,2,52738,最终幻想世界,13,30,1,1,2,True,True,...,2.0,1.5,2.0,1.5,2.0,1.5,1.5,1.5,1.0,1.0
7,3,64062,谢文东2,18,129,2,2,2,False,True,...,2.0,1.0,NaN,NaN,NaN,NaN,1.5,2.0,1.0,1.5
8,3,69067,谢文东2,14,188,1,2,2,True,False,...,1.0,2.0,NaN,NaN,NaN,NaN,1.5,1.0,2.0,1.5
9,3,119148,电视剧,8,60,4,3,8,False,False,...,7.0,4.5,2.0,2.0,4.5,4.5,4.5,5.5,7.0,7.5


In [22]:
# target encoding
stat_data['type'] = stat_data.apply(lambda x:kb[x['kb_id']]['type'][0],axis=1)
data_groupbytype = stat_data.groupby('type',as_index=False)['label'].agg({
    'type_label_mean':'mean','type_label_count':'count'})

In [23]:
data_ctr['type'] = data_ctr.apply(lambda x:kb[x['kb_id']]['type'][0],axis=1)

In [24]:
data_ctr = data_ctr.merge(data_groupbytype,on='type',how='left')

In [25]:
data_ctr.head()

,text_id,kb_id,train_mention,num_attrs,num_abstract_words,num_alias,m_id,num_candidates,mention_equal_subject,shuminghao,...,rank_label_count,rank_m_label_mean,rank_m_label_count,rank_num_candidates,rank_mention_equal_subject,rank_shuminghao,rank_entity_common,type,type_label_mean,type_label_count
0,1,119304,忏悔录,13,158,1,0,5,True,True,...,3.0,1.0,3.0,3.0,3.0,3.0,3.0,creativework,0.080972,278850
1,1,38456,忏悔录,11,347,4,0,5,True,True,...,3.0,4.5,3.0,3.0,3.0,3.0,3.0,communicationmedium,0.080189,78851
2,1,386314,忏悔录,14,226,2,0,5,True,True,...,3.0,4.5,3.0,3.0,3.0,3.0,3.0,communicationmedium,0.080189,78851
3,1,113234,忏悔录,14,360,8,0,5,True,True,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,creativework,0.080972,278850
4,1,372331,忏悔录,5,31,1,0,5,True,True,...,3.0,2.0,3.0,3.0,3.0,3.0,3.0,creativework,0.080972,278850


In [26]:
# rank 特征
to_rank = ['num_attrs', 'num_abstract_words', 'num_alias', 'label_mean', 'label_count', 'm_label_mean', 
             'm_label_count','num_candidates','mention_equal_subject','shuminghao','entity_common',
           'type_label_mean'
          ]
for f in to_rank:
    data_group = data_ctr.groupby('m_id')[f].rank(ascending=False)
    data_ctr['rank_'+f] = data_group

In [27]:
# mean and var
to_rank = ['num_attrs', 'num_abstract_words', 'num_alias', 'label_mean', 'label_count', 'm_label_mean', 
             'm_label_count','num_candidates','mention_equal_subject','shuminghao','entity_common',
           'type_label_mean'
          ]
for f in to_rank:
    data_group = data_ctr.groupby('m_id')[f].agg({f+'_var':'var',f+'_mean':'mean',f+'_max':'max'})
    data_ctr = data_ctr.merge(data_group,on='m_id',how='left')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  import sys


In [31]:
from sklearn.externals import joblib
gbm_pickle = joblib.load('model_link/lgb.pkl')

In [32]:
predictor = ['num_attrs',
 'num_abstract_words',
 'num_alias',
 'num_candidates',
 'mention_equal_subject',
 'shuminghao',
 'entity_common',
 'len_mention',
 'mention_start',
 'type_label_mean',
 'type_label_count',
 'label_mean',
 'label_count',
 'm_label_mean',
 'm_label_count',
 'rank_num_attrs',
 'rank_num_abstract_words',
 'rank_num_alias',
 'rank_label_mean',
 'rank_label_count',
 'rank_m_label_mean',
 'rank_m_label_count',
 'rank_num_candidates',
 'rank_mention_equal_subject',
 'rank_shuminghao',
 'rank_entity_common',
 'rank_type_label_mean',
 'num_attrs_var',
 'num_attrs_mean',
 'num_attrs_max',
 'num_abstract_words_var',
 'num_abstract_words_mean',
 'num_abstract_words_max',
 'num_alias_var',
 'num_alias_mean',
 'num_alias_max',
 'label_mean_var',
 'label_mean_mean',
 'label_mean_max',
 'label_count_var',
 'label_count_mean',
 'label_count_max',
 'm_label_mean_var',
 'm_label_mean_mean',
 'm_label_mean_max',
 'm_label_count_var',
 'm_label_count_mean',
 'm_label_count_max',
 'num_candidates_var',
 'num_candidates_mean',
 'num_candidates_max',
 'mention_equal_subject_var',
 'mention_equal_subject_mean',
 'mention_equal_subject_max',
 'shuminghao_var',
 'shuminghao_mean',
 'shuminghao_max',
 'entity_common_var',
 'entity_common_mean',
 'entity_common_max',
 'type_label_mean_var',
 'type_label_mean_mean',
 'type_label_mean_max']

# predictor = ['num_attrs',
#        'num_abstract_words', 'num_alias', 'num_candidates',
#        'mention_equal_subject', 'shuminghao', 'len_mention', 'mention_start',
#         'label_mean', 'label_count', 'm_label_mean',
#        'm_label_count', 'rank_num_attrs', 'rank_num_abstract_words',
#        'rank_num_alias', 'rank_label_mean', 'rank_label_count',
#        'rank_m_label_mean', 'rank_m_label_count', 'rank_num_candidates',
#        'rank_mention_equal_subject', 'rank_shuminghao', 'rank_entity_common','entity_common']
link_grades = gbm_pickle.predict(data_ctr[predictor])

In [34]:
data_ctr[predictor].head()

,num_attrs,num_abstract_words,num_alias,num_candidates,mention_equal_subject,shuminghao,entity_common,len_mention,mention_start,type_label_mean,...,mention_equal_subject_max,shuminghao_var,shuminghao_mean,shuminghao_max,entity_common_var,entity_common_mean,entity_common_max,type_label_mean_var,type_label_mean_mean,type_label_mean_max
0,13,158,1,5,True,True,1,3,1,0.080972,...,True,0.0,1.0,True,0.0,1.0,1,1.837534e-07,0.080659,0.080972
1,11,347,4,5,True,True,1,3,1,0.080189,...,True,0.0,1.0,True,0.0,1.0,1,1.837534e-07,0.080659,0.080972
2,14,226,2,5,True,True,1,3,1,0.080189,...,True,0.0,1.0,True,0.0,1.0,1,1.837534e-07,0.080659,0.080972
3,14,360,8,5,True,True,1,3,1,0.080972,...,True,0.0,1.0,True,0.0,1.0,1,1.837534e-07,0.080659,0.080972
4,5,31,1,5,True,True,1,3,1,0.080972,...,True,0.0,1.0,True,0.0,1.0,1,1.837534e-07,0.080659,0.080972


In [36]:
gbm_pickle.feature_importance()

array([ 7633, 14110,  2480,  1162,   912,   876,  2178,  3592, 11589,
        4271,  4529,  6564,  3536,  6569,  3351,  5338,  4598,  3827,
        3285,  2274,  2994,   806,   190,  2739,  3223,  5942,  5023,
        8852,  8442,  4066,  8957,  8757,  7363,  4792,  4513,  1434,
        5852,  4331,  4099,  3469,  4062,  2894,  5100,  2374,  4071,
          21,   650,   158,     0,    25,     1,  2681,  2556,    36,
        3388,  3470,    86,  7228,  7114,  1492,  7049,  6913,  2113])

In [37]:
gbm_pickle.feature_name()

['num_attrs',
 'num_abstract_words',
 'num_alias',
 'num_candidates',
 'mention_equal_subject',
 'shuminghao',
 'entity_common',
 'len_mention',
 'mention_start',
 'type_label_mean',
 'type_label_count',
 'label_mean',
 'label_count',
 'm_label_mean',
 'm_label_count',
 'rank_num_attrs',
 'rank_num_abstract_words',
 'rank_num_alias',
 'rank_label_mean',
 'rank_label_count',
 'rank_m_label_mean',
 'rank_m_label_count',
 'rank_num_candidates',
 'rank_mention_equal_subject',
 'rank_shuminghao',
 'rank_entity_common',
 'rank_type_label_mean',
 'num_attrs_var',
 'num_attrs_mean',
 'num_attrs_max',
 'num_abstract_words_var',
 'num_abstract_words_mean',
 'num_abstract_words_max',
 'num_alias_var',
 'num_alias_mean',
 'num_alias_max',
 'label_mean_var',
 'label_mean_mean',
 'label_mean_max',
 'label_count_var',
 'label_count_mean',
 'label_count_max',
 'm_label_mean_var',
 'm_label_mean_mean',
 'm_label_mean_max',
 'm_label_count_var',
 'm_label_count_mean',
 'm_label_count_max',
 'num_candida

In [20]:
# # test　train data acc
# raw_train_data = pd.read_pickle('features/1_stat_feature.pkl')
# raw_train_data['link_grade'] = gbm_pickle.predict(raw_train_data[predictor])
# max_id = raw_train_data.groupby('m_id',as_index=False)['link_grade'].agg({'max_id':'idxmax'})['max_id']
# valid_train_data = raw_train_data.loc[max_id,:]
# valid_train_data['label'].mean()

In [38]:
link_grades

array([7.61461434e-01, 7.41668554e-03, 5.00891135e-03, ...,
       1.05712488e-05, 4.45147928e-06, 9.96729653e-01])

In [39]:
data_ctr['link_grade'] = link_grades

In [40]:
print(data_ctr[data_ctr.text_id==996]['entity_common'])

17917    1
17918    1
Name: entity_common, dtype: int64


In [296]:
# index_df = val_data.groupby('m_id')['index'].apply(list)
# label_package = val_data.groupby('m_id',as_index=False)['label'].agg({'label_id_max':'idxmax'})['label_id_max']

In [61]:
submit['m_id'].nunique()

25737

In [41]:
# 过滤低预测值数据
valid_data = data_ctr[data_ctr.link_grade>0.4]
submit = valid_data.groupby('m_id',as_index=False)['link_grade'].agg({'label_id_max':'idxmax'})
m = submit.shape[0]


In [42]:
hit = valid_data.loc[submit['label_id_max'],:]

In [43]:
print(hit.head())
valid_data = hit

    text_id   kb_id train_mention  num_attrs  num_abstract_words  num_alias  \
0         1  119304           忏悔录         13                 158          1   
5         2  303980        最终幻想世界         12                 132          1   
7         3   64062          谢文东2         18                 129          2   
16        3  202048           电视剧         14                 426         18   
17        4   79540            进口          9                  68          1   

    m_id  num_candidates  mention_equal_subject  shuminghao  ...  \
0      0               5                   True        True  ...   
5      1               2                   True       False  ...   
7      2               2                  False        True  ...   
16     3               8                   True       False  ...   
17     4               1                   True       False  ...   

    shuminghao_var  shuminghao_mean  shuminghao_max  entity_common_var  \
0         0.000000            1.000       

In [297]:
# max_id = data_ctr.groupby('m_id',as_index=False)['link_grade'].agg({'max_id':'idxmax'})['max_id']
# sum(ner_result['pred'].apply(lambda x:len(x)))
# valid_data = data_ctr.loc[max_id,:]

In [27]:
new_train_data[0]

{'text_id': 1,
 'text': '《忏悔录》 20111016 荒唐出轨',
 'mention_data': [{'mention': '忏悔录', 'offset': '1'}]}

In [262]:
# 原生数据
raw_submit = []
text_idto_text = {}
with open('data/raw_data/train.json', 'r') as f:
    for line in tqdm(f):
        s = json.loads(line)
        text_idto_text[s['text_id']] = s['text']
        raw_submit.append(s)
        
kbid_finder = {}
for index,row in valid_data.iterrows():
    kbid_finder[str(row['text_id'])+row['train_mention']] = str(row['kb_id'])
    
submit_data = []
for item in new_train_data:
    # if item['kb_id']=='nil':continue
    item['text_id']=str(item['text_id'])
    item['text']=text_idto_text[item['text_id']]
    m_list = []
    for m in item['mention_data']:
        m['kb_id']=kbid_finder[str(item['text_id'])+m['mention']]
        m['mention'] = item['text'][int(m['offset']):int(m['offset'])+len(m['mention'])]

90000it [00:00, 104515.98it/s]


In [263]:
submit_data = new_train_data

In [265]:
len(submit_data)

10000

In [227]:
new_train_data[42]

{'text_id': 80043,
 'text': '正在观看《玉蒲团ⅲ官人我要》高清-战地影院',
 'mention_data': [{'mention': '玉蒲团Ⅲ官人我要', 'offset': '5', 'kb_id': '114822'}]}

In [223]:
len(submit_data)

42

In [266]:
# ner f1 评测
m = 0
n = 0
hit = 0
for i in range(10000):
    try:
        assert submit_data[i]['text']==raw_submit[i+80000]['text']
    except:
        print(i)
        print( submit_data[i]['text'],raw_submit[i+80000]['text'])
    # assert submit_data[i]['text']==raw_submit[i]['text']
    m+=len(submit_data[i]['mention_data'])
    # print(submit_data[i],raw_submit[i+8000])
    for item in submit_data[i]['mention_data']:
        for s_item in raw_submit[i+80000]['mention_data']:
            if s_item['kb_id']=='NIL':continue
            # print(item['mention'],s_item['mention'])
            if item['mention'] == s_item['mention']:
                hit+=1
                break
    for s_item in raw_submit[i+80000]['mention_data']:
        if s_item['kb_id'] !='NIL':
            n+=1
print(m,n,hit)
print(hit/m,hit/n)

26812 28513 22543
0.8407802476503058 0.7906218216252235


In [267]:
# f1 评测
m = 0
n = 0
hit = 0
for i in range(10000):
    try:
        assert submit_data[i]['text']==raw_submit[i+80000]['text']
    except:
        print( submit_data[i]['text'],raw_submit[i+80000]['text'])
    #assert submit_data[i]['text']==raw_submit[i]['text']
    m+=len(submit_data[i]['mention_data'])
    #n+=len(raw_submit[i+80000]['mention_data'])
    for item in submit_data[i]['mention_data']:
        if item in raw_submit[i+80000]['mention_data']:
            hit+=1
    for s_item in raw_submit[i+80000]['mention_data']:
        if s_item['kb_id'] != 'NIL':
            n+=1
print(m,n,hit)
print(hit/m,hit/n)

26812 28513 21214
0.7912128897508578 0.744011503524708


In [108]:
raw_submit[80000:]

[{'text_id': '80001',
  'text': '张学友一首《当爱变成习惯》很经典',
  'mention_data': [{'kb_id': '374536', 'mention': '张学友', 'offset': '0'},
   {'kb_id': '198575', 'mention': '当爱变成习惯', 'offset': '6'}]},
 {'text_id': '80002',
  'text': '《高庸武侠小说全集》在线阅读',
  'mention_data': [{'kb_id': '36336', 'mention': '高庸武侠小说全集', 'offset': '1'}]},
 {'text_id': '80003',
  'text': '《大话仙侠》三大职业新人必备职业_大话仙',
  'mention_data': [{'kb_id': '53519', 'mention': '大话仙侠', 'offset': '1'},
   {'kb_id': '39794', 'mention': '职业', 'offset': '8'},
   {'kb_id': '231880', 'mention': '新人', 'offset': '10'},
   {'kb_id': '39794', 'mention': '职业', 'offset': '14'}]},
 {'text_id': '80004',
  'text': '《轮回:三界至尊》_黔首的刀著_玄幻',
  'mention_data': [{'kb_id': 'NIL', 'mention': '轮回:三界至尊', 'offset': '1'},
   {'kb_id': 'NIL', 'mention': '黔首的刀', 'offset': '10'}]},
 {'text_id': '80005',
  'text': '中国音乐学院 江南 男中音 《隐衷》《黑雾》《 ...',
  'mention_data': [{'kb_id': '194401', 'mention': '中国音乐学院', 'offset': '0'},
   {'kb_id': 'NIL', 'mention': '江南', 'offset': '7'},
   {'kb

In [101]:
submit_data

[{'text_id': '80001',
  'text': '张学友一首《当爱变成习惯》很经典',
  'mention_data': [{'kb_id': '374536', 'mention': '张学友', 'offset': '0'},
   {'kb_id': '198575', 'mention': '当爱变成习惯', 'offset': '6'}]},
 {'text_id': '80002',
  'text': '《高庸武侠小说全集》在线阅读',
  'mention_data': [{'kb_id': '36336', 'mention': '高庸武侠小说全集', 'offset': '1'}]},
 {'text_id': '80003',
  'text': '《大话仙侠》三大职业新人必备职业_大话仙',
  'mention_data': [{'kb_id': '53519', 'mention': '大话仙侠', 'offset': '1'},
   {'kb_id': '103811', 'mention': '职业', 'offset': '8'},
   {'kb_id': '231880', 'mention': '新人', 'offset': '10'},
   {'kb_id': '103811', 'mention': '职业', 'offset': '14'}]},
 {'text_id': '80004', 'text': '《轮回:三界至尊》_黔首的刀著_玄幻', 'mention_data': []},
 {'text_id': '80005',
  'text': '中国音乐学院 江南 男中音 《隐衷》《黑雾》《 ...',
  'mention_data': [{'kb_id': '194401', 'mention': '中国音乐学院', 'offset': '0'},
   {'kb_id': '237640', 'mention': '男中音', 'offset': '10'}]},
 {'text_id': '80006', 'text': '为毛德莱文不出破败。。_德莱文吧', 'mention_data': []},
 {'text_id': '80007',
  'text': '《重生之最强王

In [31]:
new_train_data

[{'text_id': 1,
  'text': '《忏悔录》 20111016 荒唐出轨',
  'mention_data': [{'mention': '忏悔录', 'offset': '1'}]},
 {'text_id': 2,
  'text': '《最终幻想世界》上线steam 11月22日发售',
  'mention_data': [{'mention': '最终幻想世界', 'offset': '1'}]},
 {'text_id': 3,
  'text': '《谢文东2》电视剧_全集(1-28集)高清在线观看',
  'mention_data': [{'mention': '谢文东2', 'offset': '1'},
   {'mention': '电视剧', 'offset': '6'}]},
 {'text_id': 4,
  'text': '进口喜宝奶粉有适合3岁以上的吗',
  'mention_data': [{'mention': '进口', 'offset': '0'},
   {'mention': '喜宝奶粉', 'offset': '2'},
   {'mention': '适合', 'offset': '7'}]},
 {'text_id': 5,
  'text': '“3·11”东日本大地震7周年悼念 仍有约七万人过着...',
  'mention_data': [{'mention': '东日本大地震', 'offset': '6'}]},
 {'text_id': 6,
  'text': '《机会来了》 - 完整版-原创-高清视频',
  'mention_data': [{'mention': '机会来了', 'offset': '1'},
   {'mention': '视频', 'offset': '18'}]},
 {'text_id': 7,
  'text': '关于gosick结局的疑惑。_gosick吧',
  'mention_data': [{'mention': '结局', 'offset': '8'}]},
 {'text_id': 8,
  'text': '【评】关于《奇幻加冕礼》自己的看法_snh48吧',
  'mention_data': [{'mention': '

In [44]:
testfile_name = 'data/raw_data/develop.json'
raw_submit = []
text_idto_text = {}
with open(testfile_name, 'r') as f:
    for line in tqdm(f):
        s = json.loads(line)
        text_idto_text[s['text_id']] = s['text']
        raw_submit.append(s)


10000it [00:00, 58588.01it/s]


In [45]:
kbid_finder = {}
for index,row in valid_data.iterrows():
    kbid_finder[str(row['text_id'])+row['train_mention']] = str(row['kb_id'])

In [47]:
new_train_data[0]

{'text_id': 1,
 'text': '《忏悔录》 20111016 荒唐出轨',
 'mention_data': [{'mention': '忏悔录', 'offset': '1'}]}

In [49]:
submit_data = []
for item in new_train_data:
    # if item['kb_id']=='nil':continue
    item['text_id']=str(item['text_id'])
    item['text']=text_idto_text[item['text_id']]
    for m in item['mention_data']:
        try:
            m['kb_id']=kbid_finder[str(item['text_id'])+m['mention']]
        except:
            m['kb_id'] = 'NIL' # 去掉某些ner
        m['mention'] = item['text'][int(m['offset']):int(m['offset'])+len(m['mention'])]
submit_data = new_train_data

In [50]:
new_submit = []
for item in submit_data:
    new_item = {}
    new_item['text_id'] = item['text_id']
    new_item['text'] = item['text']
    new_item['mention_data'] = []
    for m in item['mention_data']:
        if m['kb_id']=='NIL':continue
        new_item['mention_data'].append(m)
    new_submit.append(new_item)

In [51]:
name_id['麦词']

KeyError: '麦词'

In [52]:
new_submit

[{'text_id': '1',
  'text': '《忏悔录》 20111016 荒唐出轨',
  'mention_data': [{'mention': '忏悔录', 'offset': '1', 'kb_id': '119304'}]},
 {'text_id': '2',
  'text': '《最终幻想世界》上线steam 11月22日发售',
  'mention_data': [{'mention': '最终幻想世界', 'offset': '1', 'kb_id': '303980'}]},
 {'text_id': '3',
  'text': '《谢文东2》电视剧_全集(1-28集)高清在线观看',
  'mention_data': [{'mention': '谢文东2', 'offset': '1', 'kb_id': '64062'},
   {'mention': '电视剧', 'offset': '6', 'kb_id': '202048'}]},
 {'text_id': '4',
  'text': '进口喜宝奶粉有适合3岁以上的吗',
  'mention_data': [{'mention': '进口', 'offset': '0', 'kb_id': '79540'},
   {'mention': '喜宝奶粉', 'offset': '2', 'kb_id': '357348'},
   {'mention': '适合', 'offset': '7', 'kb_id': '382390'}]},
 {'text_id': '5',
  'text': '“3·11”东日本大地震7周年悼念 仍有约七万人过着...',
  'mention_data': [{'mention': '东日本大地震', 'offset': '6', 'kb_id': '20128'}]},
 {'text_id': '6',
  'text': '《机会来了》 - 完整版-原创-高清视频',
  'mention_data': [{'mention': '机会来了', 'offset': '1', 'kb_id': '219277'},
   {'mention': '视频', 'offset': '18', 'kb_id': '150546

In [54]:
data_ctr[['text_id','kb_id','train_mention','label_mean','label_count','m_label_mean',
                                 'm_label_count','link_grade']]

,text_id,kb_id,train_mention,label_mean,label_count,m_label_mean,m_label_count,link_grade
0,1,119304,忏悔录,0.833333,18.0,0.833333,18.0,0.761461
1,1,38456,忏悔录,0.000000,18.0,0.000000,18.0,0.007417
2,1,386314,忏悔录,0.000000,18.0,0.000000,18.0,0.005009
3,1,113234,忏悔录,0.055556,18.0,0.055556,18.0,0.147793
4,1,372331,忏悔录,0.111111,18.0,0.111111,18.0,0.033161
5,2,303980,最终幻想世界,1.000000,5.0,1.000000,5.0,0.960504
6,2,52738,最终幻想世界,0.000000,5.0,0.000000,5.0,0.040160
7,3,64062,谢文东2,NaN,NaN,NaN,NaN,0.684556
8,3,69067,谢文东2,NaN,NaN,NaN,NaN,0.117827
9,3,119148,电视剧,0.042180,4220.0,0.000340,2945.0,0.000004


In [47]:
for id in name_id['老子']:
    print(kb[id])
    print('\n')

{'alias': ['老子', '老聃'], 'subject_id': '234094', 'subject': '老子讲经台', 'type': ['human'], 'data': [{'predicate': '摘要', 'object': '老子2000多年前，在楼观台写下了5000字的《道德经》，并在这里讲经论道，开创了我国重要的宗教——道教，为中华民族留下了一笔重要的精神财富，《道德经》是中华文化的精髓。楼观台就成了老子的讲经台了。'}, {'predicate': '出生地', 'object': '苦县'}, {'predicate': '主要作品', 'object': '道德经'}, {'predicate': '所处时代', 'object': '春秋时代'}, {'predicate': '本名', 'object': '老子'}, {'predicate': '字号', 'object': '伯阳'}, {'predicate': '民族族群', 'object': '汉族'}, {'predicate': '别称', 'object': '老聃'}, {'predicate': '主要成就', 'object': '道家学派创始人, 辩证法思想、无为而治、性命双修'}, {'predicate': '去世时间', 'object': '前471年'}, {'predicate': '出生时间', 'object': '约前571年(待考)'}, {'predicate': '义项描述', 'object': '老子讲经台'}, {'predicate': '标签', 'object': '出版物'}, {'predicate': '标签', 'object': '文化'}, {'predicate': '标签', 'object': '人物'}]}


{'alias': ['老子'], 'subject_id': '360360', 'subject': '老子', 'type': ['thing'], 'data': [{'predicate': '摘要', 'object': '老子，又称老聃、李耳，是我国古代伟大的哲学家和思想家、道家学派创始人。其被唐皇武后封为太上老君，世界文化名人，世界百位历史名人之一，存世有《道德经》(又

In [151]:

# pickle.dump(submit_data,open('submit/result.pkl','wb'))

In [55]:
import json
with open('submit/result_6.json', 'w') as f:
    for line in new_submit:
        json.dump(line,f,ensure_ascii=False)
        f.write('\n')

In [59]:
c = 0
for item in new_submit:
    c += len(item['mention_data'])
print(c)

25758


In [40]:
previous = pickle.load(open('submit/result.pkl','rb'))